# Creating Analytic Dataset

**STEPS**:  
1. Merging neighborhood-level house price dataset with demographics dataset. 
2. Merging neighborhood-level house price and house value index dataset with neighborhood-level school quality dataset.
3. *Merging with demographics dataset*


In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Prepare School Dataset

In [73]:
df_school_location = pd.read_csv("./data/schools_neighborhoods.csv")
df_school_attendance = pd.read_csv("./data/cleaned_school_attendance.csv")
df_school_attendance.head()

,DBN,school_name,year,pct_attendance,pct_chronically_absent
0,01M015,P.S. 015 Roberto Clemente,2019,92.624725,24.479166
1,01M015,P.S. 015 Roberto Clemente,2020,92.891335,28.431372
2,01M015,P.S. 015 Roberto Clemente,2021,87.973442,42.211056
3,01M015,P.S. 015 Roberto Clemente,2022,84.995102,59.473682
4,01M015,P.S. 015 Roberto Clemente,2023,87.266937,50.485435


In [74]:
df_school_location_clean = df_school_location[
    [
        "system_code",
        "location_name",
        "Managed_by_name",
        "location_type_description",
        "Location_Category_Description",
        "open_date",
        "Status_descriptions",
        "primary_address_line_1",
        "Community_district",
        "Council-district",
        "Census_tract",
        'NTA_Name',
        'NTA2020',
        'NTAName',
        'BoroName'
    ]
]

df_school_location_clean.columns = [
    "DBN",
    "school_name",
    "school_type",
    "academic_type",
    "grade_type",
    "open_date",
    "status",
    "address",
    "community_district",
    "council_district",
    "census_tract",
    "NTA_Name",
    "NTA2020",
    "NTA2020_Name",
    "Borough",
]

In [75]:
df_school = pd.merge(df_school_location_clean, df_school_attendance, left_on="DBN", right_on="DBN", how="left")
df_school = df_school.drop(columns=["school_name_y", "NTA_Name","NTA2020_Name","Borough"])
df_school.rename(columns={"school_name_x": "school_name"}, inplace=True)

In [76]:
df_school.columns

Index(['DBN', 'school_name', 'school_type', 'academic_type', 'grade_type',
       'open_date', 'status', 'address', 'community_district',
       'council_district', 'census_tract', 'NTA2020', 'year', 'pct_attendance',
       'pct_chronically_absent'],
      dtype='object')

## Merge House Sales Dataset with Demographics Dataset

In [77]:
df_sales = pd.read_csv("./data/cleaned_neighbor_sales.csv")
df_neighborhood_demo = pd.read_csv("./data/census_demo.csv")
df_neighborhood_demo.drop(columns=["Unnamed: 0", "tract", "NTAName"], inplace=True)
df_neighborhood_demo.rename(columns={
    "Total Estimate": "total_population",
    "White alone Estimate": "white",
    "Black or African American alone Estimate": "black",
    "American Indian and Alaska Native alone Estimate": "american_indian_alaska_native",
    "Asian alone Estimate": "asian",
    "Native Hawaiian and Other Pacific Islander alone Estimate": "hawaiian_pacific_islander",
    "Some Other Race alone Estimate": "other_race",
    "Two or More Races Estimate": "two_or_more",
    "Two races including Some Other Race Estimate": "two_include_other",
    "Two races excluding Some Other Race, and three or more races Estimate": "two_other",
    "Median Income In The Past 12 Months (In 2020 Inflation-Adjusted Dollars)": "median_income",
}, inplace=True)

In [78]:
df_neighborhood_demo['multiple_race'] = df_neighborhood_demo['two_or_more'] + df_neighborhood_demo['two_include_other'] + df_neighborhood_demo['two_other']
race_cols = ['white', 'black', 'american_indian_alaska_native', 'asian', 'hawaiian_pacific_islander','multiple_race', 'other_race']
for race in race_cols:
    race_pct = race+"_pct"
    # Round to 3 decimal places
    df_neighborhood_demo[race_pct] = np.round(df_neighborhood_demo[race] / df_neighborhood_demo['total_population']*100,3)
df_neighborhood_demo.drop(columns=['two_or_more', 'two_include_other', 'two_other']+race_cols, inplace=True)

In [79]:
df_sales.columns

Index(['nta_code', 'nta_name', 'dof_name', 'neighborhood', 'borough', 'year',
       'number_of_sales', 'average_sale_price', 'median_sale_price',
       'lowest_sale_price', 'highest_sale_price', 'hvi'],
      dtype='object')

In [80]:
df_neighborhood_demo.columns

Index(['GEOID', 'NTA2020', 'total_population', 'median_income', 'white_pct',
       'black_pct', 'american_indian_alaska_native_pct', 'asian_pct',
       'hawaiian_pacific_islander_pct', 'multiple_race_pct', 'other_race_pct'],
      dtype='object')

In [81]:
df_house = pd.merge(df_sales, df_neighborhood_demo, left_on="nta_code", right_on="NTA2020", how="left")

## Merge School Dataset with House Price and House Value Index Dataset

In [82]:
school_neighbor_merge = pd.merge(df_school, df_house, left_on=["NTA2020","year"], right_on=["nta_code","year"], how="left")

In [83]:
school_neighbor_merge.drop(columns=["dof_name","neighborhood","NTA2020_x","NTA2020_y"], inplace=True)
school_neighbor_merge.columns
# print(school_neighbor_merge.describe())

Index(['DBN', 'school_name', 'school_type', 'academic_type', 'grade_type',
       'open_date', 'status', 'address', 'community_district',
       'council_district', 'census_tract', 'year', 'pct_attendance',
       'pct_chronically_absent', 'nta_code', 'nta_name', 'borough',
       'number_of_sales', 'average_sale_price', 'median_sale_price',
       'lowest_sale_price', 'highest_sale_price', 'hvi', 'GEOID',
       'total_population', 'median_income', 'white_pct', 'black_pct',
       'american_indian_alaska_native_pct', 'asian_pct',
       'hawaiian_pacific_islander_pct', 'multiple_race_pct', 'other_race_pct'],
      dtype='object')

## Merge School/HVI with Evictions

In [84]:
## read in evictions at tract level aggregate
df_evictions = pd.read_csv("data/nyc_tract_evictions_aggregate.csv.gz")
df_evictions.columns

Index(['YEAR', 'GEOID', 'EVICTION_COUNT'], dtype='object')

In [89]:
# check missingness of GEOID and year for each dataset
print(df_evictions[['GEOID','YEAR']].isnull().sum())
print(school_neighbor_merge[['GEOID','year']].isnull().sum())

GEOID    0
YEAR     0
dtype: int64
GEOID    0
year     0
dtype: int64


In [90]:
# prep for merge
# convert school_neighbor_merge['GEOID'] to int64
school_neighbor_merge = school_neighbor_merge.dropna(subset=['GEOID'])

school_neighbor_merge['GEOID'] = school_neighbor_merge['GEOID'].astype('int64')

# merge evictions data
school_sales_evictions = pd.merge(school_neighbor_merge, df_evictions, left_on=["GEOID","year"], right_on=["GEOID","YEAR"], how="left")
school_sales_evictions.columns

Index(['DBN', 'school_name', 'school_type', 'academic_type', 'grade_type',
       'open_date', 'status', 'address', 'community_district',
       'council_district', 'census_tract', 'year', 'pct_attendance',
       'pct_chronically_absent', 'nta_code', 'nta_name', 'borough',
       'number_of_sales', 'average_sale_price', 'median_sale_price',
       'lowest_sale_price', 'highest_sale_price', 'hvi', 'GEOID',
       'total_population', 'median_income', 'white_pct', 'black_pct',
       'american_indian_alaska_native_pct', 'asian_pct',
       'hawaiian_pacific_islander_pct', 'multiple_race_pct', 'other_race_pct',
       'YEAR', 'EVICTION_COUNT'],
      dtype='object')

In [91]:
# school_neighbor_merge.to_csv("./data/analytic_dataset.csv", index=False)
school_sales_evictions.to_csv("./data/analytic_dataset.csv", index=False)